## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
from IPython.display import clear_output

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nome,US,64.50,-165.41,12.20,85,1,4.61,clear sky
1,1,Brunsbuttel,DE,53.90,9.15,63.00,93,100,8.01,overcast clouds
2,2,Bluff,NZ,-46.60,168.33,55.99,68,27,8.99,scattered clouds
3,3,Kapaa,US,22.08,-159.32,75.00,100,90,9.17,light rain
4,4,Barrow,US,71.29,-156.79,26.60,68,75,12.75,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 12
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nome,US,64.50,-165.41,12.20,85,1,4.61,clear sky
1,1,Brunsbuttel,DE,53.90,9.15,63.00,93,100,8.01,overcast clouds
2,2,Bluff,NZ,-46.60,168.33,55.99,68,27,8.99,scattered clouds
3,3,Kapaa,US,22.08,-159.32,75.00,100,90,9.17,light rain
4,4,Barrow,US,71.29,-156.79,26.60,68,75,12.75,broken clouds
...,...,...,...,...,...,...,...,...,...,...
703,703,Ono,JP,34.85,134.93,62.60,93,75,12.75,light intensity shower rain
708,708,Hofn,IS,64.25,-15.21,35.47,76,0,13.60,clear sky
709,709,Outlook,CA,51.50,-107.05,46.40,60,76,8.05,broken clouds
710,710,Dossor,KZ,47.53,52.98,53.83,41,0,15.93,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                483
City                   483
Country                479
Lat                    483
Lng                    483
Max Temp               483
Humidity               483
Cloudiness             483
Wind Speed             483
Current Description    483
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.count()

City_ID                479
City                   479
Country                479
Lat                    479
Lng                    479
Max Temp               479
Humidity               479
Cloudiness             479
Wind Speed             479
Current Description    479
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nome,US,12.20,clear sky,64.50,-165.41,
1,Brunsbuttel,DE,63.00,overcast clouds,53.90,9.15,
2,Bluff,NZ,55.99,scattered clouds,-46.60,168.33,
3,Kapaa,US,75.00,light rain,22.08,-159.32,
4,Barrow,US,26.60,broken clouds,71.29,-156.79,
6,Bredasdorp,ZA,64.40,overcast clouds,-34.53,20.04,
8,Hermanus,ZA,64.00,overcast clouds,-34.42,19.23,
10,Carnarvon,AU,73.40,clear sky,-24.87,113.63,
11,Krasnoselkup,RU,22.33,overcast clouds,65.70,82.47,
12,Mataura,NZ,55.99,scattered clouds,-46.19,168.86,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
counter=0
index_2=0
# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #Add lat and lng to params dictionary
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search.
    response=requests.get(base_url,params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        print(f'Success {index_2} of {len(hotel_df)} | {"{:.2f}".format((index_2/len(hotel_df)*100))}%')
        counter+=1
        index_2+=1
        if counter == 10:
            clear_output()
            counter=0
    except IndexError:
        print(f'\nFaliure {index_2} of {len(hotel_df)} | {"{:.2f}".format((index_2/len(hotel_df.index)*100))}% \n\t\tSkipping...\n')
        counter+=1
        index_2+=1
        if counter == 10:
            clear_output()
            counter=0
        pass
print("-"*30+"\nCompleted\t100.00%\n"+"-"*30)

Success 470 of 479 | 98.12%

Faliure 471 of 479 | 98.33% 
		Skipping...

Success 472 of 479 | 98.54%
Success 473 of 479 | 98.75%
Success 474 of 479 | 98.96%
Success 475 of 479 | 99.16%
Success 476 of 479 | 99.37%
Success 477 of 479 | 99.58%
Success 478 of 479 | 99.79%
------------------------------
Completed	100.00%
------------------------------


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   479
Country                479
Max Temp               479
Current Description    479
Lat                    479
Lng                    479
Hotel Name             479
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
clean_hotel_df["Max Temp"].head()

0    12.20
1    63.00
2    55.99
3    75.00
4    26.60
Name: Max Temp, dtype: float64

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))